In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
import argparse
import json
import numpy as np
import pandas as pd

from src.config import cfg
from src.fold_train import run_kfold_training
from src.ensemble import stacked_predict

In [ ]:
import sys

if "ipykernel" in sys.modules:
    sys.argv = ["run_train_and_predict.ipynb",
                "--mode", "predict",
                "--model_out_dir", "outputs/20250922_104022"]  # to avoid argparse error

parser = argparse.ArgumentParser()
parser.add_argument("--mode", choices=["train", "predict", "train_then_predict"], default="train_then_predict")
parser.add_argument("--test_csv", default=None, help="Override cfg.test_csv for prediction")
parser.add_argument("--models_out_dir", default=None, help="If provided, use this trained models root for prediction")
parser.add_argument("--save_preds", default="predictions.csv", help="Where to save predictions in predict modes")
args = parser.parse_args()

out_dir = args.models_out_dir

if args.mode in ["train", "train_then_predict"]:
    out_dir = run_kfold_training()
    print("Artifacts at:", out_dir)

if args.mode in ["predict", "train_then_predict"]:
    test_csv = args.test_csv or cfg.test_csv
    if test_csv is None or not os.path.exists(test_csv):
        raise ValueError(f"Test CSV not found: {test_csv}")
    if out_dir is None or not os.path.exists(out_dir):
        raise ValueError("Missing models_out_dir to run prediction")

    preds, label_names = stacked_predict(test_csv, out_dir)

    df_test = pd.read_csv(test_csv)
    if "id" in df_test.columns:
        sub = pd.DataFrame({"id": df_test["id"].values, "label": [label_names[i] for i in preds]})
    else:
        sub = pd.DataFrame({"label": [label_names[i] for i in preds]})

    sub_path = os.path.join(out_dir, args.save_preds)
    sub.to_csv(sub_path, index=False)
    print(f"Saved predictions to {sub_path}")

    np.save(os.path.join(out_dir, "predictions.npy"), preds)
    with open(os.path.join(out_dir, "label_names.json"), "w") as f:
        json.dump({"label_names": label_names}, f, indent=2)
print(f"Saved preds npy and label names under {out_dir}")

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobe

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not tuple